In [2]:
# A későbbiekben felhasznált csomagok importálása

from openai import OpenAI
import yaml
import requests
from bs4 import BeautifulSoup
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

In [3]:
# A HTML-ben ajax hívások is lehetnek, amelyek további kommenteket töltenek be
# Az ajax hívások kommentjeit itt gyűjtöm össze

def parse_ajax_comments(post_id, total_ajax_comments, last_parent):
    ajax_comments = []
    offset = 1
    ajax_url = "https://kiszamolo.hu/wp-admin/admin-ajax.php"
    while len(ajax_comments) < total_ajax_comments:
        payload = {
            "action": "wpdLoadMoreComments",
            "sorting": "oldest",
            "offset": offset,
            "lastParentId": last_parent,
            "isFirstLoad": 0,
            "wpdType": "post",
            "postId": post_id,
        }
        response = requests.post(ajax_url, data=payload)
        html = response.json()["data"]['comment_list']
        soup = BeautifulSoup(html, 'html.parser')
        comment_divs = soup.find_all("div", class_="wpd-comment")
        for div in comment_divs:
            text_div = div.find("div", class_="wpd-comment-text")
            comment = text_div.get_text(strip=True) if text_div else None
            ajax_comments.append(comment)
        last_parent = comment_divs[-1].get("id").replace("wpd-comm-", "").split('_')[0] if comment_divs else 0
        offset += 1
    return ajax_comments

In [4]:
# Az url alapján parse-olom a kommenteket
# Ha kevesebb komment van, mint amennyit a wpd-thread-info class mutat, akkor ajax hívásokkal jön a többi
# (az újabb cikkek esetében már nincsenek ajax hívások, de korábban voltak)

def parse_comments(url):
    response = requests.get(url)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')
    info_div = soup.find("div", class_="wpd-thread-info")
    total_comments = int(info_div["data-comments-count"]) if info_div else 0
    comment_divs = soup.find_all("div", class_="wpd-comment")
    comments=[]
    for div in comment_divs:
        text_div = div.find("div", class_="wpd-comment-text")
        comment = text_div.get_text(strip=True) if text_div else None
        comments.append(comment)
    if len(comments) >= total_comments:
        return comments  
    else:
        shortlink = soup.find("link", rel="shortlink") # A postId lekérése ajaxhoz
        post_id = None
        if shortlink and "?p=" in shortlink["href"]:
            post_id = shortlink["href"].split("?p=")[-1]
        last_parent = comment_divs[-1].get("id").replace("wpd-comm-", "").split('_')[0] if comment_divs else 0 # Last_parent lekérése ajaxhoz
        total_ajax_comments = total_comments - len(comments)
        ajax_comments = parse_ajax_comments(post_id, total_ajax_comments, last_parent)
        return comments + ajax_comments

In [5]:
# Ezek a függvények a megadott keresőszó és oldalszám alapján visszaadják a cikkek címét, dátumát,  url-jét, tatralmát
# Az alapstringbe (ezek lesznek az opciók a widgetben) dátum és a cím kerül
# A valueba pedig egy tuple kerül, hogy a widgetben kezelni tudjam a HTML tartalmat is és a linket is külön

def build_html(content, title, link):
    return (f"<div style='margin: 0; padding: 0; line-height:1.4; font-family: Arial, sans-serif; font-size: 14px;text-align: justify;'>"
            f"<h4 style='margin: 0 0 6px 0;'>{title.upper()}</h4>"
            f"{content}<br><br>"
            f"<a href='{link}' target='_blank' style='color: #2980b9; text-decoration: none;'>{link}</a>"
            f"</div>")

def get_urls_from_kiszamolo_page(page, search_text):
    result = []
    url = f"https://kiszamolo.hu/page/{page}/?s={search_text}" if page > 1 else f"https://kiszamolo.hu/?s={search_text}"    
    response = requests.get(url)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')
    articles = soup.find_all('article', class_='oxy-post')
    for article in articles:
        a_tag = article.find('a', class_='oxy-post-title')
        title = a_tag.text.strip()
        link = a_tag['href']
        # Dátum
        date_div = article.find('div', class_='oxy-post-meta-date')
        date = date_div.text.strip() if date_div else 'N/A'
        # Tartalom
        content_div = article.find('div', class_='oxy-post-content')
        content = content_div.text.strip() if content_div else 'N/A'
        html_content = build_html(content, title, link)
        content_tuple = (html_content, link)
        result.append((f"{date}  |  {title}", content_tuple))
    return result

**Copy this text to auth.yaml and fill in your own OpenAI API key** <br>
openai_api_key: "YOUR_OPENAI_API_KEY_HERE"

In [6]:
# Az OpenAI prompt összeállítása, majd az API meghívása az összegfoglaláshoz

credentials = yaml.load(open('auth.yaml'), Loader=yaml.FullLoader)
OPENAI_APIKEY = credentials.get('OPENAI_APIKEY')

def ask_ai(comments_list, verbosity):
    comments = "\n".join(comments_list) #listából szöveggé alakítjuk a kommenteket
    message = f""" 
    Itt van egy cikkhez érkezett kommentek listája. 
    Foglaljad össze a főbb véleményeket, úgy, hogy:

    1. A főbb vélemények számozott listába szedve jelenjenek meg (<ol><li>…</li></ol>), ne pontokban.
    2. NE tegyél semmilyen kódblokk jelölést (pl. ```html).
    2. Szerepeljenek konkrét számok, ha a kommentelők említenek (pl. árak, százalékok, mennyiségek).
    3. Jelezd, ha többen ugyanazt a számot vagy adatot írják.
    4. Maximum {verbosity} pont legyen, ha kevés komment van, lehet rövidebb.
    
    Kommentek: {comments}
    """
    client = OpenAI(api_key=OPENAI_APIKEY)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": message}])
    return response.choices[0].message.content

In [7]:
# IPyWidgets objektumok létrehozása és felrakása a képernyőre
# Valamint a widgetek eseményeinek kezelése

# Keresőmező widget
search_input = widgets.Text(
    value = '',
    placeholder='Keresés cikkek között...',
    description='Keresőszó:',
    layout=widgets.Layout(width='350px'))

# Keresőgomb widget
search_button = widgets.Button(
    description='🔍',
    tooltip= 'Keresés',
    layout=widgets.Layout(width='40px'))

# Keresőgomb eseménye
def search(_=None):
    page_selector.value = 1
    page = page_selector.value
    search_text = search_input.value.strip()
    article_selector.options = get_urls_from_kiszamolo_page(page, search_text)
    
search_button.on_click(search)

# Oldalválasztó
page_selector = widgets.IntText(
    value=1,
    description='Oldal:',
    layout=widgets.Layout(width='150px'))

# Oldalszám változásakor a cikklista frissítése
def on_page_change(change):
    if change['new'] < 1:
        page_selector.value = 1  
    article_selector.options = get_urls_from_kiszamolo_page(page_selector.value, search_input.value.strip())

# Figyeli az oldalválasztó értékének változását
page_selector.observe(on_page_change, names='value')

# Cikkválasztó widget
article_selector = widgets.Select(
    options= get_urls_from_kiszamolo_page(page_selector.value, search_input.value.strip()),
    description='Cikkek:',
    rows= 15,
    layout=widgets.Layout(width='740px'))
                            
def on_select_change(change):
    textbox.value = change['new'][0] or ''
    ai_response.value = ''

# Figyeli a cikkválasztó értékének változását
article_selector.observe(on_select_change, names='value')

# HTML widget, itt jelenik meg a cikk rövid tartalma
textbox = widgets.HTML(
        layout=widgets.Layout(margin='20px 0 20px 90px', width = '650px'),
        value=article_selector.value[0])

# Gomb az összefoglaláshoz
summarize_button = widgets.Button(
    description="Kérem a kommentek összefoglalását",
    button_style='primary',
    layout=widgets.Layout(width="250px", margin='0 0 0 90px'))

# Ezzel a csúszkával lehet állítani, hogy hány pontban válaszoljon az LLM
verbose_slider = widgets.IntSlider(
    min=4,
    max=25, 
    value=10,
    description="Összefoglaló tömörsége:",
    style={'description_width': '230px', 
        'handle_color': 'blue'},
    layout=widgets.Layout(
        width='450px',
        height='40px'))

# Ha kattintunk a gombra, parse-oljuk a kiválasztott oldalról a kommenteket, és átadjuk egy listában az ask_ai függvénynek (ha nem üres a lista)
# Átadjuk az ask_ai()-nak tömörségre vonatkozó slider értékét is
# Az eredményeket megjelenítjük egy HTML widgetsben
def on_summarize_click(b):
    style_title = "font-size: 16px; font-weight: bold;"
    style_body = "margin: 5px 0 0 0; padding: 0; line-height:1.4; font-family: Arial, sans-serif; font-size: 14px;text-align: justify;"
    selected_article_link = article_selector.value[1]
    ai_response.value = f"<div style='{style_body}'>Openai is working on comments for {selected_article_link}</div>"
    comments = parse_comments(selected_article_link)
    if not comments:
        ai_response.value = f"<div style='{style_body}'>A cikkhez nincsenek hozzászólások</div>"
    else:         
        response = ask_ai(comments, verbose_slider.value)
        ai_response.value = f"""
                              <div style="{style_title}">Kommentek összefoglalása::</div>
                              <div style="{style_body}">{response}</div>                          
                            """        
                                           
summarize_button.on_click(on_summarize_click)

# Itt fog megjelenni az OpenAi válasza
ai_response = widgets.HTML(
        layout=widgets.Layout(margin='20px 0 20px 90px', width='650px'))

# A widgetek megjelenítése
search_box = widgets.HBox([search_input, search_button])
display(search_box, article_selector, page_selector, textbox, verbose_slider, summarize_button, ai_response)

Select(description='Cikkek:', layout=Layout(width='740px'), options=(('2025-08-31  |  A ChatGPT szerint így ke…

IntText(value=1, description='Oldal:', layout=Layout(width='150px'))

HTML(value="<div style='margin: 0; padding: 0; line-height:1.4; font-family: Arial, sans-serif; font-size: 14p…

IntSlider(value=10, description='Összefoglaló tömörsége:', layout=Layout(height='40px', width='450px'), max=25…

Button(button_style='primary', description='Kérem a kommentek összefoglalását', layout=Layout(margin='0 0 0 90…

HTML(value='', layout=Layout(margin='20px 0 20px 90px', width='650px'))